# Quickstart LSST Verify Demo

<br>Owner: **Keith Bechtol** ([@bechtol](https://github.com/LSSTScienceCollaborations/StackClub/issues/new?body=@bechtol))
<br>Last Verified to Run: **2019-01-15**
<br>Verified Stack Release: **d_2019_01_15**, need to check v16.0

LSST Verify python pakcage: https://github.com/lsst/verify

This notebook is based on the documentation at https://sqr-019.lsst.io/

Another example: https://github.com/mareuter/notebooks/blob/master/LSST/Systems_Engineering/System_Verification_SQuaSH/System_Verification_Demo.ipynb

for which the metrics are defined here:
https://github.com/mareuter/notebooks/tree/master/LSST/Systems_Engineering/System_Verification_SQuaSH

In [1]:
# What version of the Stack am I using?
! echo $HOSTNAME
! eups list -s | grep lsst_distrib

jld-lab-kbechtol-d20190115
lsst_distrib          16.0-2-g3466de9+33 	current d_2019_01_15 setup


In [2]:
import lsst.verify

In [3]:
METRIC_PACKAGE = "verify_demo"
metrics = lsst.verify.MetricSet.load_metrics_package(METRIC_PACKAGE)
specs = lsst.verify.SpecificationSet.load_metrics_package(METRIC_PACKAGE)

In [4]:
metrics

Name,Description,Units,Reference,Tags
str17,str28,str15,str27,str16
demo.ZeropointRMS,Photometric calibration RMS.,$\mathrm{mmag}$,https://example.com/PhotRMS,"demo, photometry"


In [5]:
specs

Name,Test,Tags
str25,str27,str7
demo.ZeropointRMS.minimum,$x$ <= 20.0 $\mathrm{mmag}$,minimum
